In [1]:
# Importar Bibliotecas

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
# Variáveis do Experimento

NUM_USERS = 100000
NUM_DAYS = 30
START_DATE = datetime(2025,1,1)

In [3]:
# Gerar um pequeno fator de "melhora" ou "piora" aleatória para o grupo de teste
# View Page -> Add to Cart: O fator será um número aleatório entre -0.01 (piora de 1%) e 0.05 (melhora de 5%)
# Add to Cart -> Purchase: O fator será um número aleatório entre -0.02 (piora de 2%) e 0.08 (melhora de 8%)

random_lift_add_to_cart = random.uniform(-0.01, 0.05)
random_lift_purchase = random.uniform(-0.02, 0.08)

In [4]:
# Definir as probabilidades base

BASE_PROBABILITIES = {
    'view_page': 1.0,
    'add_to_cart': 0.10,  #10% dos usuários que visualizam, adicionam ao carrinho (média de mercado)
    'purchase': 0.25  #25% dos usuários que adicionam, compram (média de mercado)
}

In [5]:
# Probabilidades de conversão para cada grupo
# O grupo de teste recebe um "lift" aleatório

PROBABILITIES = {
    'control': BASE_PROBABILITIES,
    'test': {
        'view_page': 1.0,
        'add_to_cart': BASE_PROBABILITIES['add_to_cart'] + random_lift_add_to_cart,
        'purchase': BASE_PROBABILITIES['purchase'] + random_lift_purchase
    }
}

In [6]:
# Probabilidades de atributos de usuários e produtos

DEVICE_PROBS = [0.7, 0.25, 0.05]
DEVICE_TYPES = ['Desktop', 'Mobile', 'Tablet']
TRAFFIC_SOURCE_PROBS = [0.4, 0.3, 0.15, 0.15]
TRAFFIC_SOURCES = ['Organic', 'Paid', 'Email', 'Social']
LOCATIONS = ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Porto Alegre','Curitiba','Salvador']
PRODUCT_CATEGORIES = ['Eletrônico', 'Moda', 'Casa', 'Livros', 'Esportes', 'Bebês', 'Automóveis', 'Beleza', 'Alimentício','Animais','Jogos']

In [7]:
# Número de produtos no nosso catálogo

NUM_PRODUCTS = 1000

In [8]:
# Vamos ver quais foram as probabilidades geradas para o grupo de teste
print("Probabilidades do Grupo de Teste (geradas aleatoriamente):")
print(f"  - Add to Cart: {PROBABILITIES['test']['add_to_cart']:.4f}")
print(f"  - Purchase: {PROBABILITIES['test']['purchase']:.4f}")

Probabilidades do Grupo de Teste (geradas aleatoriamente):
  - Add to Cart: 0.1467
  - Purchase: 0.2947


In [9]:
# Geração de IDs únicos para eventos
event_id_counter = 0
def generate_event_id():
    global event_id_counter
    event_id_counter += 1
    return event_id_counter

def simulate_user_journey(user_id, group, start_time, session_id):
    """
    Simula a jornada de um único usuário através do funil de vendas.
    """
    events = []
    
    # Probabilidades de conversão para o grupo do usuário
    group_probs = PROBABILITIES[group]
    
    # Geração de atributos aleatórios para o usuário
    device = np.random.choice(DEVICE_TYPES, p=DEVICE_PROBS)
    traffic_source = np.random.choice(TRAFFIC_SOURCES, p=TRAFFIC_SOURCE_PROBS)
    location = random.choice(LOCATIONS)
    
    # Passo 1: O usuário visualiza a página do produto
    if random.random() < group_probs['view_page']:

        # Simula o produto visualizado
        product_id = random.randint(1, NUM_PRODUCTS)
        product_category = random.choice(PRODUCT_CATEGORIES)
        product_price = round(random.uniform(20.0, 300.0), 2)
        
        event = {
            'event_id': generate_event_id(),
            'user_id': user_id,
            'group': group,
            'timestamp': start_time,
            'event_type': 'view_page',
            'session_id': session_id,
            'product_id': product_id,
            'product_category': product_category,
            'product_price': product_price,
            'quantity': None,
            'order_id': None,
            'total_value': None,
            'device_type': device,
            'traffic_source': traffic_source,
            'location': location
        }
        events.append(event)
        
        # Passo 2: O usuário adiciona ao carrinho (probabilidade dependente do grupo)
        if random.random() < group_probs['add_to_cart']:
            add_to_cart_time = start_time + timedelta(seconds=random.randint(30, 600))
            
            event = {
                'event_id': generate_event_id(),
                'user_id': user_id,
                'group': group,
                'timestamp': add_to_cart_time,
                'event_type': 'add_to_cart',
                'session_id': session_id,
                'product_id': product_id,
                'product_category': product_category,
                'product_price': product_price,
                'quantity': 1,
                'order_id': None,
                'total_value': None,
                'device_type': device,
                'traffic_source': traffic_source,
                'location': location
            }
            events.append(event)
            
            # Passo 3: O usuário finaliza a compra (probabilidade dependente do grupo)
            if random.random() < group_probs['purchase']:
                purchase_time = add_to_cart_time + timedelta(seconds=random.randint(60, 1200))
                order_id = random.randint(100000, 999999)
                total_value = round(product_price * 1.1, 2)
                
                event = {
                    'event_id': generate_event_id(),
                    'user_id': user_id,
                    'group': group,
                    'timestamp': purchase_time,
                    'event_type': 'purchase',
                    'session_id': session_id,
                    'product_id': product_id,
                    'product_category': product_category,
                    'product_price': product_price,
                    'quantity': 1,
                    'order_id': order_id,
                    'total_value': total_value,
                    'device_type': device,
                    'traffic_source': traffic_source,
                    'location': location
                }
                events.append(event)
                
    return events

In [ ]:
# Lista para armazenar todos os eventos gerados
all_events = []

# Gerando dados para uma amostra de 5 usuários
for user_id in range(1, 6):
    # Atribuição aleatória de grupo para o teste A/B
    group = np.random.choice(['control', 'test'], p=[0.5, 0.5])
    
    # Geração de uma data e hora de início de sessão aleatória
    session_start_time = START_DATE + timedelta(days=random.randint(0, NUM_DAYS-1),
                                                 hours=random.randint(0, 23),
                                                 minutes=random.randint(0, 59))
    session_id = random.randint(1000, 9999) # ID de sessão aleatório

    # Simula a jornada do usuário e adiciona os eventos à lista
    user_events = simulate_user_journey(user_id, group, session_start_time, session_id)
    all_events.extend(user_events)

# Converte a lista de dicionários para um DataFrame do Pandas
df_sample = pd.DataFrame(all_events)

# Exibe o resultado para validação
print("Amostra do DataFrame (primeiros 5 usuários):")
display(df_sample)

Amostra do DataFrame (primeiros 5 usuários):


,event_id,user_id,group,timestamp,event_type,session_id,product_id,product_category,product_price,quantity,order_id,total_value,device_type,traffic_source,location
0,1,1,test,2025-01-01 02:33:00,view_page,5971,417,Jogos,110.84,NaN,None,None,Desktop,Organic,Belo Horizonte
1,2,2,test,2025-01-04 23:56:00,view_page,8577,718,Beleza,174.70,NaN,None,None,Desktop,Organic,Salvador
2,3,3,test,2025-01-13 13:54:00,view_page,5302,88,Animais,186.45,NaN,None,None,Desktop,Organic,Belo Horizonte
3,4,4,test,2025-01-26 08:03:00,view_page,8923,250,Livros,26.76,NaN,None,None,Desktop,Paid,Curitiba
4,5,5,test,2025-01-15 02:29:00,view_page,2534,201,Beleza,244.04,NaN,None,None,Desktop,Paid,Salvador
5,6,5,test,2025-01-15 02:38:30,add_to_cart,2534,201,Beleza,244.04,1.0,None,None,Desktop,Paid,Salvador


In [11]:
# Lista para armazenar todos os eventos gerados
all_events = []

# Gerando os dados para todos os  100.000 usuários
for user_id in range(1, NUM_USERS + 1):
    group = np.random.choice(['control', 'test'], p=[0.5, 0.5])
    session_start_time = START_DATE + timedelta(days=random.randint(0, NUM_DAYS - 1),
                                                hours=random.randint(0, 23),
                                                minutes=random.randint(0, 59))
    
    session_id = random.randint(100000, 999999)

    user_events = simulate_user_journey(user_id, group, session_start_time, session_id)
    all_events.extend(user_events)

# Cria o DataFrame final
df = pd.DataFrame(all_events)

# Exibe as primeiras linhas e o tamanho total do DataFrame
print("DataFrame Final:")
display(df.head())
print(f"\nNúmero total de eventos gerados: {len(df)}")

DataFrame Final:


,event_id,user_id,group,timestamp,event_type,session_id,product_id,product_category,product_price,quantity,order_id,total_value,device_type,traffic_source,location
0,7,1,control,2025-01-06 17:28:00,view_page,602554,938,Casa,225.92,NaN,NaN,NaN,Mobile,Organic,Rio de Janeiro
1,8,2,test,2025-01-08 14:08:00,view_page,709878,701,Jogos,217.76,NaN,NaN,NaN,Desktop,Organic,Salvador
2,9,3,control,2025-01-06 03:59:00,view_page,148334,161,Jogos,106.27,NaN,NaN,NaN,Desktop,Paid,Salvador
3,10,4,control,2025-01-18 01:33:00,view_page,291347,239,Automóveis,195.19,NaN,NaN,NaN,Desktop,Social,Belo Horizonte
4,11,4,control,2025-01-18 01:42:43,add_to_cart,291347,239,Automóveis,195.19,1.0,NaN,NaN,Desktop,Social,Belo Horizonte



Número total de eventos gerados: 115754


In [12]:
# Salva o DataFrame em um arquivo CSV

df.to_csv('C:/Users/natal/nataliaguarnieri/proj_1_otimizacao_ecommerce_testeAB/data/ecommerce_events_data.csv', index=False)

print("\nBase de dados salva com sucesso como 'ecommerce_events_data.csv'")


Base de dados salva com sucesso como 'ecommerce_events_data.csv'


In [13]:
# Carrega a base de dados do arquivo CSV

df = pd.read_csv('C:/Users/natal/nataliaguarnieri/proj_1_otimizacao_ecommerce_testeAB/data/ecommerce_events_data.csv')

In [14]:
# Sanity Check

# 1. Chega a distribuição de usuários nos grupos
user_counts = df.groupby('group')['user_id'].nunique()
print("Número de usuários únicos por grupo:")
print(user_counts)
print("\n" + "-"*30 + "\n")

Número de usuários únicos por grupo:
group
control    49865
test       50135
Name: user_id, dtype: int64

------------------------------



In [15]:
# 2. Chega a contagem total de eventos
event_counts = df['event_type'].value_counts()
print("Contagem total de eventos por tipo:")
print(event_counts)
print("\n" + "-"*30 + "\n")

Contagem total de eventos por tipo:
event_type
view_page      100000
add_to_cart     12337
purchase         3417
Name: count, dtype: int64

------------------------------



In [16]:
# 3. Contagem de eventos por tipo para cada grupo
event_counts_by_group = df.groupby(['group', 'event_type']).size().unstack(fill_value=0)
print("Contagem de eventos por tipo e por grupo:")
print(event_counts_by_group)

Contagem de eventos por tipo e por grupo:
event_type  add_to_cart  purchase  view_page
group                                       
control            5175      1298      49865
test               7162      2119      50135
